### A1 — Add repo path and verify imports
Append `../python` to `sys.path` so imports resolve to the repository’s `python/` package. Then attempt to import `models.base_transformer.CMRTransformer` to verify setup.


In [1]:
# A2 — Add repo python path and verify core import
import os, sys
import torch
from types import SimpleNamespace
import pandas as pd
import re

REPO_PY_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "python"))
if REPO_PY_PATH not in sys.path:
    sys.path.append(REPO_PY_PATH)

try:
    from models.base_transformer import CMRTransformer  # noqa: F401
    print("CMRTransformer import: OK")
except Exception as e:
    print("CMRTransformer import: FAILED", e)


DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda"
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    DEVICE = "mps"
print("DEVICE:", DEVICE)

REPO_PY_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", "python"))

/Users/montraydavis/contextual-memory-reweaving/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


CMRTransformer import: OK
DEVICE: mps


## Config

In [2]:
ADAPTER_NAME = "gemma"
MODEL_NAME = "google/gemma-3-4b-it"

TARGET_LAYERS = [2, 4]
BUFFER_SIZE = 20

# Build a tiny tokenizer and optional HF tokenizer
USE_HF_TOKENIZER = True  # flip to True to use HuggingFace tokenizer when available

# Optional HF tokenizer
hf_tokenizer = None

# Tiny tokenizer: whitespace split with a growing vocab
toy_vocab = {"<pad>": 0, "<unk>": 1}

# Minimal validation
if not TARGET_LAYERS:
    raise ValueError("At least one target layer is required")
if BUFFER_SIZE < 1:
    raise ValueError("BUFFER_SIZE must be >= 1")

print({"TARGET_LAYERS": TARGET_LAYERS, "BUFFER_SIZE": BUFFER_SIZE})

{'TARGET_LAYERS': [2, 4], 'BUFFER_SIZE': 20}


## Dataset

In [14]:
SCENARIO = [
  {"text": "Home page: company overview, featured products, testimonials, and call-to-action buttons for sign-up.", "privacy": False},
  {"text": "About page: company history, mission statement, team bios, and office locations with contact details.", "privacy": False},
  {"text": "Products page: catalog of available services with pricing tiers, feature comparisons, and demo videos.", "privacy": False},
  {"text": "Contact page: contact form, phone numbers, email addresses, live chat widget, and business hours.", "privacy": False},
  {"text": "Blog page: articles, tutorials, industry news, and SEO-optimized content with comment sections.", "privacy": False},
  {"text": "Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.", "privacy": False},
  {"text": "FAQ page: common questions organized by category with search functionality and helpful resources.", "privacy": False},
  {"text": "Terms of Service page: legal agreements, user responsibilities, limitation of liability, and dispute resolution.", "privacy": False},
  {"text": "Privacy Policy page: data collection practices, cookie usage, third-party integrations, and user rights.", "privacy": False},
  {"text": "Careers page: open positions, company culture, benefits overview, and application submission portal.", "privacy": False},
  {"text": "Login page: user authentication with email/password, social login options, and forgot password recovery.", "privacy": False},
  {"text": "Register page: new user sign-up form with email verification, terms acceptance, and account activation.", "privacy": False},
  {"text": "User Dashboard: personalized homepage showing account status, recent activity, and quick action buttons.", "privacy": False},
  {"text": "Profile Settings: user information management, password changes, notification preferences, and account deletion.", "privacy": False},
  {"text": "Billing page: payment methods, invoice history, subscription management, and upgrade/downgrade options.", "privacy": False},
  {"text": "Order History: past purchases, order tracking, return requests, and downloadable receipts.", "privacy": False},
  {"text": "Shopping Cart: selected items, quantity adjustments, coupon codes, and checkout initiation.", "privacy": False},
  {"text": "Checkout page: shipping information, payment processing, order review, and purchase confirmation.", "privacy": False},
  {"text": "Search Results: product/content listings with filters, sorting options, and pagination controls.", "privacy": False},
  {"text": "Help Center: knowledge base articles, video tutorials, troubleshooting guides, and ticket submission.", "privacy": False},
  {"text": "API Documentation: endpoint references, authentication methods, code examples, and rate limiting info.", "privacy": False},
  {"text": "Downloads page: software installers, user manuals, templates, and resource files with version history.", "privacy": False},
  {"text": "News page: press releases, company announcements, media coverage, and investor relations updates.", "privacy": False},
  {"text": "Events page: upcoming webinars, conferences, product launches, and registration forms.", "privacy": False},
  {"text": "Partners page: integration listings, affiliate program details, and partner application process.", "privacy": False},
  {"text": "Sitemap page: hierarchical website structure with all public pages listed and organized by category.", "privacy": False},
  {"text": "404 Error page: custom not found message with navigation suggestions and search functionality.", "privacy": False},
  {"text": "Maintenance page: scheduled downtime notification with estimated completion time and status updates.", "privacy": False},
  {"text": "Security page: data protection measures, compliance certifications, and security best practices.", "privacy": False},
  {"text": "Mobile App page: download links for iOS/Android, feature highlights, and screenshot gallery.", "privacy": False},
  {"text": "Integrations page: third-party service connections, setup instructions, and compatibility matrix.", "privacy": False},
  {"text": "Admin Dashboard: system overview, user management controls, analytics data, and configuration settings. Contains administrative functions.", "privacy": True},
  {"text": "User Management: admin interface for viewing user accounts, permissions, suspension controls, and personal data. Contains PII.", "privacy": True},
  {"text": "Analytics Panel: detailed usage statistics, user behavior tracking, conversion metrics, and sensitive business data.", "privacy": True},
  {"text": "Database Admin: direct database access, query execution, backup management, and sensitive system data.", "privacy": True},
  {"text": "Financial Reports: revenue data, transaction details, tax information, and confidential business metrics.", "privacy": True},
  {"text": "Employee Portal: staff directory, salary information, performance reviews, and internal communications. Contains PII.", "privacy": True},
  {"text": "System Logs: server logs, error tracking, security events, and debugging information with user data.", "privacy": True},
  {"text": "Configuration Manager: system settings, API keys, database credentials, and infrastructure management.", "privacy": True},
  {"text": "Backup Interface: data recovery tools, backup scheduling, restore functions, and system maintenance access.", "privacy": True},
  {"text": "Security Center: threat monitoring, access logs, vulnerability scans, and incident response tools.", "privacy": True},
  {"text": "Payment Processing: transaction management, refund processing, merchant settings, and financial data access.", "privacy": True},
  {"text": "Customer Support Admin: ticket management, user impersonation, chat monitoring, and PII access for support.", "privacy": True},
  {"text": "Marketing Analytics: campaign performance, customer segments, A/B test results, and behavioral data.", "privacy": True},
  {"text": "Content Management: CMS backend, publishing controls, media library, and editorial workflow management.", "privacy": True},
  {"text": "API Management: endpoint monitoring, rate limiting controls, usage analytics, and developer key management.", "privacy": True},
  {"text": "Audit Trail: compliance logging, user activity tracking, data access records, and regulatory reporting.", "privacy": True},
  {"text": "HR Management: employee records, hiring pipeline, payroll data, and confidential personnel information. Contains PII.", "privacy": True},
  {"text": "Legal Documents Manager: contracts, NDAs, compliance files, and confidential legal correspondence.", "privacy": True},
  {"text": "Vendor Management: supplier contracts, payment processing, vendor onboarding, and procurement data.", "privacy": True},
  {"text": "Executive Dashboard: board reports, strategic metrics, competitive analysis, and confidential business intelligence.", "privacy": True},
  {"text": "Data Export Tool: bulk data extraction, customer information downloads, and sensitive data migration utilities. Contains PII.", "privacy": True},
  {"text": "Email Campaign Manager: subscriber lists, email templates, delivery tracking, and customer communication data.", "privacy": True},
  {"text": "Inventory Management: stock levels, supplier information, cost data, and warehouse operations interface.", "privacy": True},
  {"text": "Customer Service Tools: case management, escalation procedures, customer history access, and support metrics.", "privacy": True},
  {"text": "Quality Assurance: testing environments, bug tracking, performance monitoring, and development tools access.", "privacy": True},
  {"text": "Compliance Dashboard: regulatory reporting, audit preparation, policy management, and risk assessment tools.", "privacy": True}
]

STOPWORDS = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"
]

def parse_page(text: str) -> str:
    # Heuristic: everything before " page" or first colon
    if " page" in text.lower():
        return text.split(" page", 1)[0].strip().title()
    return text.split(":", 1)[0].strip().title()

rows = []
for i, it in enumerate(SCENARIO, 1):
    rows.append({
        "Step": i,
        "Privacy": "Private" if it["privacy"] else "Public",
        "Page": parse_page(it["text"]),
        "Description": it["text"],
    })

## Transformer

In [4]:
# C1 — Instantiate CMRTransformer

# Minimal config object with only required fields
base_cfg = SimpleNamespace(vocab_size=50257, num_hidden_layers=8)

memory_config = {
    "target_layers": TARGET_LAYERS,
    "buffer_size": BUFFER_SIZE,
}

cmr = CMRTransformer(config=base_cfg, memory_config=memory_config)
cmr.to(DEVICE)

# Ensure nested layers move too (workaround for SimpleNamespace)
if hasattr(cmr, "transformer") and hasattr(cmr.transformer, "h"):
    for lyr in cmr.transformer.h:
        lyr.to(DEVICE)

print("CMRTransformer ready on", DEVICE)

CMRTransformer ready on mps


# Target Layer Validation

In [5]:
# C2 — Validate target layers
num_layers = len(cmr.transformer.h)
invalid = [idx for idx in TARGET_LAYERS if idx < 0 or idx >= num_layers]
print({"num_layers": num_layers, "TARGET_LAYERS": TARGET_LAYERS})
if invalid:
    print("Warning: some target layers are out of range:", invalid)
else:
    print("All target layers are valid.")

{'num_layers': 8, 'TARGET_LAYERS': [2, 4]}
All target layers are valid.


## Register Hooks

In [6]:
# C3 — Register memory hooks
cmr.register_memory_hooks()
active_hook_layers = sorted(list(cmr.layer_hooks.keys()))
print("Active hook layers:", active_hook_layers)

Active hook layers: [2, 4]


## Encoder / Decoder

In [7]:
def toy_encode(text: str):
    tokens = text.strip().split()
    ids = []
    for tok in tokens:
        if tok not in toy_vocab:
            toy_vocab[tok] = len(toy_vocab)
        ids.append(toy_vocab.get(tok, toy_vocab["<unk>"]))
    return ids

def toy_decode(ids):
    inv_vocab = {i: t for t, i in toy_vocab.items()}
    return " ".join(inv_vocab.get(i, "<unk>") for i in ids)

def encode(text: str) -> torch.LongTensor:
    if hf_tokenizer is not None:
        ids = hf_tokenizer.encode(text, add_special_tokens=False)
    else:
        ids = toy_encode(text)
    if len(ids) == 0:
        ids = [toy_vocab.get("<unk>", 1)]
    return torch.tensor(ids, dtype=torch.long, device=DEVICE).unsqueeze(0)  # [1, T]

def decode(ids: torch.LongTensor) -> str:
    arr = ids.detach().cpu().view(-1).tolist()
    if hf_tokenizer is not None:
        return hf_tokenizer.decode(arr)
    return toy_decode(arr)


## Setup Tokenizer

In [8]:
if USE_HF_TOKENIZER:
    try:
        from transformers import AutoTokenizer
        hf_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        print("Using HF tokenizer for:", MODEL_NAME)
    except Exception as e:
        print("Falling back to toy tokenizer (HF load failed):", e)
        hf_tokenizer = None
else:
    print("Using toy tokenizer (default)")

Using HF tokenizer for: google/gemma-3-4b-it


## Memory Search

In [9]:
# E3 — Storyboard (emoji)
for r in rows:
    lock = "🔒" if r["Privacy"] == "Private" else "🔓"
    print(f"{lock} Step {r['Step']}: {r['Page']} — {r['Description']}")


# I2 — Score positions on the last run's hidden states
# Reuse the last call's enhanced hidden states if available
try:
    last_hidden = outputs.get("last_hidden_state")  # [1, T, H]
except Exception:
    last_hidden = None

if last_hidden is None:
    # Fallback: run one more step to get hidden states
    tmp = cmr(encode(SCENARIO[-1]["text"]))
    last_hidden = tmp.get("last_hidden_state")

# Simple variance-based scoring over hidden dim
scores = torch.var(last_hidden, dim=-1).squeeze(0)  # [T]
# Normalize to [0,1]
if scores.numel() > 0:
    scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-8)

# Show top-5 positions
k = min(5, scores.numel())
vals, idxs = torch.topk(scores, k)
print("Top positions:")
for rank in range(k):
    print(f"#{rank+1}: pos={int(idxs[rank])}, score={float(vals[rank]):.3f}")

🔓 Step 1: Home — Home page: company overview, featured products, testimonials, and call-to-action buttons for sign-up.
🔓 Step 2: About — About page: company history, mission statement, team bios, and office locations with contact details.
🔓 Step 3: Products — Products page: catalog of available services with pricing tiers, feature comparisons, and demo videos.
🔓 Step 4: Contact — Contact page: contact form, phone numbers, email addresses, live chat widget, and business hours.
🔓 Step 5: Blog — Blog page: articles, tutorials, industry news, and SEO-optimized content with comment sections.
🔓 Step 6: Pricing — Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.
🔓 Step 7: Faq — FAQ page: common questions organized by category with search functionality and helpful resources.
🔓 Step 8: Terms Of Service — Terms of Service page: legal agreements, user responsibilities, limitation of liability, and dispute resolution.
🔓 Step 9: Privacy Policy — Pri

/var/folders/0t/mq3cq_z505nbzy4_cw1s4x_00000gn/T/ipykernel_13769/2660531025.py:30: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  print(f"#{rank+1}: pos={int(idxs[rank])}, score={float(vals[rank]):.3f}")


## Decode Memory Search Results

In [17]:
def _tokens(s: str):
    return set(re.findall(r"[a-z0-9]+", s.lower()))

# Map scenario index to text for retrieval
SCEN_TEXTS = [it["text"] for it in SCENARIO] if 'SCENARIO' in globals() else []

def retrieve_top_text_memories(query_text: str, k: int = 3):
    if not SCEN_TEXTS:
        return []
    q = _tokens(query_text)
    scored = []
    for idx, txt in enumerate(SCEN_TEXTS):
        t = _tokens(txt)
        j = len(q & t) / (len(q | t) + 1e-6)
        scored.append((j, idx, txt))
    scored.sort(reverse=True)
    return [txt for _, _, txt in scored[:k]]

def remove_stop_words(text: str):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

USER_QUERY = remove_stop_words("I want to see the billing page ")
context_snippets = retrieve_top_text_memories(USER_QUERY, k=3)

context_snippets

['Billing page: payment methods, invoice history, subscription management, and upgrade/downgrade options.',
 'Pricing page: subscription plans, feature matrix, annual vs monthly billing, and enterprise quotes.',
 'Compliance Dashboard: regulatory reporting, audit preparation, policy management, and risk assessment tools.']